In [7]:
import pandas as pd
import seaborn as sns
sns.set_theme(style="white")

from valentine.algorithms import Coma
from valentine.algorithms import JaccardDistanceMatcher
from valentine.algorithms import SimilarityFlooding
from valentine.algorithms import DistributionBased
from valentine.algorithms import Cupid
from valentine.metrics import F1Score, PrecisionTopNPercent
from valentine import valentine_match
import pprint

import os
import sys
import jellyfish
from polyfuzz import PolyFuzz

sys.path.append(os.path.join(os.path.abspath(''), '..'))
from gdc.gdc_api import GDCSchema 


pp = pprint.PrettyPrinter(indent=4, sort_dicts=True)


ModuleNotFoundError: No module named 'gdc_api'

In [11]:
df_dou = pd.read_csv('../data/extracted-tables/Dou_UCEC_CPTAC3_meta_table_V2.1.csv')

df_dou.head(5)

df_dou['Path_Stage_Primary_Tumor-pT'].unique()

array(['pT1a (FIGO IA)', nan, 'pT3a (FIGO IIIA)', 'pT1 (FIGO I)',
       'pT1b (FIGO IB)', 'pT2 (FIGO II)', 'pT3b (FIGO IIIB)'],
      dtype=object)

In [2]:
df_target = pd.read_csv('../data/target.csv')
df_target.drop('study', axis=1, inplace=True)
print(df_target.shape)
df_target.head(5)

(1068, 16)


,case_submitter_id,age_at_diagnosis,race,ethnicity,gender,vital_status,ajcc_pathologic_t,ajcc_pathologic_n,ajcc_pathologic_stage,tumor_grade,tumor_focality,tumor_largest_dimension_diameter,primary_diagnosis,morphology,tissue_or_organ_of_origin,tumor_code
0,01BR001,20089.0,black or african american,not hispanic or latino,female,Alive,T2,N1c,Stage II,GX,Not Reported,Not Reported,Invasive carcinoma of no special type,8500/3,"Breast, NOS",BRCA
1,01BR008,17532.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA
2,01BR009,23376.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA
3,01BR010,23741.0,black or african american,not hispanic or latino,female,Not Reported,Not Reported,Not Reported,Not Reported,GX,Not Reported,Not Reported,Not Reported,Not Reported,"Breast, NOS",BRCA
4,01BR015,12784.0,white,not hispanic or latino,female,Alive,T2,N1,Stage II,GX,Not Reported,Not Reported,Invasive carcinoma of no special type,8500/3,"Breast, NOS",BRCA


### Check GDC Dictionary for matches
#### For each column name in target, what is the "closest" entry in GDC dictionary, and calculate column value/ GDC domain intersection


In [3]:
gdc_schema = GDCCandidateMatcher()
gdc_matches = gdc_schema.parse_df(df_target)

gdc_matches_values = {entry: entry_value['candidate'] for entry, entry_value in gdc_matches.items()}
#print(f'Number of matches: {len(gdc_matches_values)} out of {len(df_target.columns)} columns in target')
gdc_matches_values

AttributeError: 'GDCSchema' object has no attribute 'parse_df'

In [22]:
for col, match in gdc_matches.items():
    if isinstance(match['values'], list):
        col_domain = set(df_target[col].unique())
        col_domain = {value.lower() for value in col_domain}
        gdc_domain = set(match['values'])
        gdc_domain = {value.lower() for value in gdc_domain}
        
        print('GDC values for ', col, gdc_domain)

        ratio_col2GDC = len(col_domain.intersection(gdc_domain)) / len(col_domain)
        ration_GDC2col = len(col_domain.intersection(gdc_domain)) / len(gdc_domain)
        print('Ratio of column values in GDC:', ratio_col2GDC)
        print('Ratio GDC values in column values :', ration_GDC2col)
        if ratio_col2GDC != 1.0:
            print('Column values that are not in GDC:', col_domain.difference(gdc_domain))
        
    else:
        print('No domain enumeration for',col,' with type', match['type'])
    print('-----------------')
            
    

No domain enumeration for case_submitter_id  with type string
-----------------
No domain enumeration for age_at_diagnosis  with type number
-----------------
GDC values for  race {'white', 'american indian or alaska native', 'asian', 'other', 'native hawaiian or other pacific islander', 'not reported', 'black or african american'}
Ratio of column values in GDC: 1.0
Ratio GDC values in column values : 0.7142857142857143
-----------------
GDC values for  ethnicity {'not hispanic or latino', 'hispanic or latino'}
Ratio of column values in GDC: 0.6666666666666666
Ratio GDC values in column values : 1.0
Column values that are not in GDC: {'not reported'}
-----------------
GDC values for  gender {'unspecified', 'unknown', 'female', 'male'}
Ratio of column values in GDC: 1.0
Ratio GDC values in column values : 0.5
-----------------
GDC values for  vital_status {'lost to follow-up', 'alive', 'dead'}
Ratio of column values in GDC: 0.6666666666666666
Ratio GDC values in column values : 0.666666

In [23]:
df_liu = pd.read_csv('../data/liuData_Supplementary_Table_1.csv')
df_liu_dictionary = pd.read_csv('../data/liuData_Data_dictionary.csv')
print(df_liu.shape)
df_liu.head(5)

(1185, 83)


,Case_ID,tumor_code,tumor_sample_id_protein,normal_sample_id_protein,is_excluded_from_pancancer_studies,reason_for_exclusion,is_gtex,specimen/aliquout_id_protein_normal,specimen/aliquout_id_protein_tumor,specimen/aliquout_id_RNA_Tumor,...,follow-up/additional_treatment_radiation_therapy_for_new_tumor,follow-up/additional_treatment_pharmaceutical_therapy_for_new_tumor,follow-up/additional_treatment_immuno_for_new_tumor,follow-up/number_of_days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor_event_loco-regional,follow-up/number_of_days_from_date_of_initial_pathologic_diagnosis_to_date_of_additional_surgery_for_new_tumor_event_metastasis,"Recurrence-free survival, days","Recurrence status (1, yes; 0, no)","Overall survival, days","Survival status (1, dead; 0, alive)","Time between collection and diagnosis, days"
0,11BR047,BRCA,11BR047-T,NaN,NaN,NaN,0.0,NaN,81116212-b7e6-454b-9579-105cf3,ef52c640-13a9-4855-9ce2-0be77a_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,369.0,0.0,NaN
1,11BR043,BRCA,11BR043-T,NaN,NaN,NaN,0.0,NaN,6d34d499-167e-42aa-9790-316fca_D2,6d34d499-167e-42aa-9790-316fca_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,374.0,0.0,NaN
2,11BR049,BRCA,11BR049-T,NaN,NaN,NaN,0.0,NaN,2e700669-85b0-43fa-a9c7-3eaf5a_D2,2e700669-85b0-43fa-a9c7-3eaf5a_D1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,379.0,0.0,NaN
3,11BR023,BRCA,11BR023-T,NaN,NaN,NaN,0.0,NaN,0a80d3c4-0758-447a-958c-ea868c,079b5600-6afc-4785-bb22-48cfab_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,398.0,0.0,NaN
4,18BR010,BRCA,18BR010-T,NaN,NaN,NaN,0.0,NaN,0bb9d596-774e-452b-9c89-a6643c_D2,0bb9d596-774e-452b-9c89-a6643c_D7_1,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,410.0,0.0,NaN


In [24]:
threshold = len(df_liu) * 0.6
print(threshold)
df_liu.dropna(thresh=threshold, axis=1, inplace=True) # remove columns with many (+50%) nulls
print(df_liu.shape)
df_liu.head(10)


711.0
(1185, 45)


,Case_ID,tumor_code,tumor_sample_id_protein,is_gtex,specimen/aliquout_id_protein_tumor,specimen/aliquout_id_RNA_Tumor,specimen/aliquout_id_DNA_Tumor,specimen/aliquout_id_DNA_Blood,Age,Sex,...,follow-up/number_of_days_from_date_of_initial_pathologic_diagnosis_to_date_of_last_contact,follow-up/adjuvant_post-operative_radiation_therapy,follow-up/adjuvant_post-operative_pharmaceutical_therapy,follow-up/adjuvant_post-operative_immunological_therapy,follow-up/measure_of_success_of_outcome_at_the_completion_of_initial_first_course_treatment,follow-up/new_tumor_after_initial_treatment,"Recurrence status (1, yes; 0, no)","Overall survival, days","Survival status (1, dead; 0, alive)","Time between collection and diagnosis, days"
0,11BR047,BRCA,11BR047-T,0.0,81116212-b7e6-454b-9579-105cf3,ef52c640-13a9-4855-9ce2-0be77a_D7_1,ef52c640-13a9-4855-9ce2-0be77a_D6_1,0bfd3ae8-d275-4dab-9360-d1df30_D1_1,59,Female,...,369.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,369.0,0.0,NaN
1,11BR043,BRCA,11BR043-T,0.0,6d34d499-167e-42aa-9790-316fca_D2,6d34d499-167e-42aa-9790-316fca_D7_1,6d34d499-167e-42aa-9790-316fca_D6_1,a1338e3d-598a-45b5-838f-9b6206_D1_1,36,Female,...,374.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,374.0,0.0,NaN
2,11BR049,BRCA,11BR049-T,0.0,2e700669-85b0-43fa-a9c7-3eaf5a_D2,2e700669-85b0-43fa-a9c7-3eaf5a_D1,2e700669-85b0-43fa-a9c7-3eaf5a_D6,ad85fadf-31b7-44f3-b6ea-73c55d_D1,45,Female,...,379.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,379.0,0.0,NaN
3,11BR023,BRCA,11BR023-T,0.0,0a80d3c4-0758-447a-958c-ea868c,079b5600-6afc-4785-bb22-48cfab_D7_1,079b5600-6afc-4785-bb22-48cfab_D6_1,70ee4720-f4dc-4b91-a6aa-c0efd9_D1_1,58,Female,...,398.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,398.0,0.0,NaN
4,18BR010,BRCA,18BR010-T,0.0,0bb9d596-774e-452b-9c89-a6643c_D2,0bb9d596-774e-452b-9c89-a6643c_D7_1,0bb9d596-774e-452b-9c89-a6643c_D6_1,feab8720-9694-4679-917f-f1287c_D1_1,41,Female,...,410.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,410.0,0.0,NaN
5,06BR003,BRCA,06BR003-T,0.0,31b13596-554e-452f-91a4-7e67a8_D2,"['31b13596-554e-452f-91a4-7e67a8_D7', 'not run']","['31b13596-554e-452f-91a4-7e67a8_D6', 'not run']","['31da4db6-8404-43f0-995d-ebc10b_D1', 'not run']",75,Female,...,323.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,323.0,0.0,NaN
6,11BR074,BRCA,11BR074-T,0.0,3367406e-d39c-4641-a3e7-44e1f3,12ce8313-6c54-4bee-a996-1aa7f8_D7,12ce8313-6c54-4bee-a996-1aa7f8_D6_1,93f961e1-fe4c-4dfc-8cbe-1dff31_D1_1,66,Female,...,367.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,367.0,0.0,NaN
7,18BR017,BRCA,18BR017-T,0.0,392b8aa4-9f99-4250-b693-326260_D2,392b8aa4-9f99-4250-b693-326260_D7_1,392b8aa4-9f99-4250-b693-326260_D6_1,f49705bd-129b-4bfe-a944-a7eb51_D1_1,72,Female,...,445.0,Not Applicable,Not Applicable,NaN,NaN,Not Applicable,0.0,445.0,0.0,NaN
8,01BR017,BRCA,01BR017-T,0.0,09659708-7747-4d59-a3b9-e221e0_D2,09659708-7747-4d59-a3b9-e221e0_D7,09659708-7747-4d59-a3b9-e221e0_D6,b76c68fc-2403-4e79-ad46-4ae336_D1,45,Female,...,413.0,Yes,Yes,NaN,NaN,No,0.0,413.0,0.0,NaN
9,06BR006,BRCA,06BR006-T,0.0,3799eeb5-c966-4059-a78f-ace269_D2,3799eeb5-c966-4059-a78f-ace269_D7,3799eeb5-c966-4059-a78f-ace269_D6,7e168c2a-f22a-44d9-ac7d-1a4c42_D1,51,Female,...,137.0,No,Yes,NaN,NaN,Yes,1.0,137.0,0.0,NaN


### Find and Analyze Keys of target and GDC

In [25]:
def find_dataframe_uniques(df):
    unique_candidates = []
    for col in df.columns:
        if df[col].nunique()/ len(df) > 0.98:
            unique_candidates.append(col)
    return unique_candidates

print(find_dataframe_uniques(df_target))
print(find_dataframe_uniques(df_liu))



['case_submitter_id']
['Case_ID']


### Find the rows in liu that were not included in GDC_formatted

In [26]:
common_values = df_target['case_submitter_id'].isin(df_liu['Case_ID']).sum()
total_values = len(df_target['case_submitter_id'])
percentage = (common_values / total_values) * 100
print(f'Percentage of comumn values between gdc_formatted and liu_et_all {percentage}')

not_in_target = df_liu[~df_liu['Case_ID'].isin(df_target['case_submitter_id'])]['Case_ID']
print(f'Values in liu_et_all not in gdc_formatted')
# print(not_in_target)
subset_not_in_target = df_liu[~df_liu['Case_ID'].isin(df_target['case_submitter_id'])]
print(subset_not_in_target)



Percentage of comumn values between gdc_formatted and liu_et_all 100.0
Values in liu_et_all not in gdc_formatted
      Case_ID tumor_code tumor_sample_id_protein  is_gtex  \
75    11BR057       BRCA               11BR057-T      0.0   
97    11BR078       BRCA               11BR078-T      0.0   
100   11BR076       BRCA               11BR076-T      0.0   
123       604       BRCA                     NaN      0.0   
124      1488       BRCA                     NaN      0.0   
...       ...        ...                     ...      ...   
1180     NX10       UCEC                     NaN      0.0   
1181     NX16       UCEC                     NaN      0.0   
1182     NX18       UCEC                     NaN      0.0   
1183     NX11       UCEC                     NaN      0.0   
1184     NX15       UCEC                     NaN      0.0   

     specimen/aliquout_id_protein_tumor specimen/aliquout_id_RNA_Tumor  \
75    8886aaca-adcf-4416-920a-794bf0_D2                            NaN   
97    

### Find rows in liu with many nulls

In [27]:
threshold = len(df_liu.columns) * 0.7
rows_with_nulls = df_liu[df_liu.isnull().sum(axis=1) > threshold]
print(f'Rows with nulls in more than {threshold} columns')
print(rows_with_nulls)

Rows with nulls in more than 31.499999999999996 columns
         Case_ID tumor_code tumor_sample_id_protein  is_gtex  \
97       11BR078       BRCA               11BR078-T      0.0   
124         1488       BRCA                     NaN      0.0   
125    100002921       BRCA                     NaN      0.0   
126    100003304       BRCA                     NaN      0.0   
127    100004012       BRCA                     NaN      0.0   
128    100004028       BRCA                     NaN      0.0   
132      05BR051       BRCA                     NaN      0.0   
133      05BR052       BRCA                     NaN      0.0   
134      05BR055       BRCA                     NaN      0.0   
135      05BR058       BRCA                     NaN      0.0   
137      18BR008       BRCA                     NaN      0.0   
351      05CO014       COAD                     NaN      0.0   
358      PT-NPJ7        GBM                     NaN      1.0   
359      PT-P44H        GBM                     

## Computes the matching between GDC_compiled file and Liu_data

In [28]:
df_target.drop('case_submitter_id', axis=1, inplace=True)
df_liu.drop('Case_ID', axis=1, inplace=True)

# matcher = Coma()
#matcher = Coma(use_instances=True)
# matcher = JaccardDistanceMatcher()
# matcher = SimilarityFlooding()
# matcher = DistributionBased()
# matcher = Cupid()

#matchers  = [Coma(),Coma(use_instances=True), JaccardDistanceMatcher(), SimilarityFlooding(), DistributionBased(), Cupid()]
matchers  = [Coma(),Coma(use_instances=True), JaccardDistanceMatcher()]

for matcher in matchers:
    print(f"Computing the matches using the {type(matcher).__name__} algorithm...")
    matches = valentine_match(df_liu, df_target, matcher)
    print("Found the following matches:")
    pp.pprint(matches)
    # print("\nOne-to-one matches:")
    # pp.pprint(matches.one_to_one())
    print('')


Computing the matches using the Coma algorithm...


Found the following matches:
{   (('table_1', 'Ethnicity'), ('table_2', 'ethnicity')): 0.7824131,
    (('table_1', 'Race'), ('table_2', 'race')): 0.7683153,
    (('table_1', 'baseline/margin_status'), ('table_2', 'vital_status')): 0.3764677,
    (('table_1', 'baseline/tumor_stage_pathological'), ('table_2', 'ajcc_pathologic_stage')): 0.4244031,
    (('table_1', 'cptac_path/histologic_grade'), ('table_2', 'ajcc_pathologic_t')): 0.37843442,
    (('table_1', 'tumor_code'), ('table_2', 'tumor_code')): 0.78438675}

Computing the matches using the Coma algorithm...
Found the following matches:
{   (('table_1', 'Ethnicity'), ('table_2', 'ethnicity')): 0.7275997,
    (('table_1', 'Race'), ('table_2', 'race')): 0.7184459,
    (('table_1', 'Sex'), ('table_2', 'gender')): 0.34026808,
    (('table_1', 'baseline/margin_status'), ('table_2', 'vital_status')): 0.3047427,
    (('table_1', 'baseline/number_of_lymph_nodes_positive_for_tumor_by_he_staining'), ('table_2', 'tumor_largest_dimension_diameter

### Checking column names between GDC and liu

In [29]:
target_cols = df_target.columns.str.lower()
liu_cols = df_liu.columns.str.lower()

for candidate in liu_cols:
    if candidate in target_cols:
        print(f"Column {candidate} is present in both datasets")
    else:
        for target in target_cols:
            if jellyfish.jaro_similarity(candidate, target) > 0.7:
                print(f"Column {candidate} matches with {target}")

Column tumor_code is present in both datasets
Column tumor_sample_id_protein matches with tumor_grade
Column tumor_sample_id_protein matches with tumor_focality
Column tumor_sample_id_protein matches with tumor_largest_dimension_diameter
Column age matches with age_at_diagnosis
Column age matches with race
Column race is present in both datasets
Column ethnicity is present in both datasets
Column cptac_path/histologic_grade matches with ajcc_pathologic_stage


In [41]:
from polyfuzz import PolyFuzz

for match,score in matches.items():
    if score > 0.3:
        print(f"Match {match} with score {score}. Showing column value matching...")

        candidate_col = match[0][1]
        target_col = (match[1][1])

        ## TODO check similarity for numerical data
        candidate_col_values = list(df_liu[candidate_col].dropna().astype(str).unique())
        target_col_values = list(df_target[target_col].dropna().astype(str).unique())

        model = PolyFuzz("TF-IDF").match(candidate_col_values, target_col_values)
        value_matches = model.get_matches()

        print(value_matches)
        print('-'*60)
        




        

Match (('table_1', 'tumor_code'), ('table_2', 'tumor_code')) with score 1.0. Showing column value matching...
    From     To  Similarity
0   BRCA   BRCA         1.0
1  ccRCC  ccRCC         1.0
2   COAD   COAD         1.0
3    GBM    GBM         1.0
4  HNSCC  HNSCC         1.0
5   LSCC   LSCC         1.0
6   LUAD   LUAD         1.0
7   HGSC   HGSC         1.0
8   PDAC   PDAC         1.0
9   UCEC   UCEC         1.0
------------------------------------------------------------
Match (('table_1', 'Race'), ('table_2', 'race')) with score 0.6666666666666666. Showing column value matching...
                               From                                To  \
0                             Asian                             asian   
1                             White                             white   
2                           Unknown                              None   
3         Black or African American         black or african american   
4  American Indian or Alaska Native  americ